# NEMU：ISA 参考

我们提出了一个类似于 Spike 的指令集模拟器 NEMU。通过优化技术，NEMU 实现了与 QEMU 类似的性能。此外，NEMU 还提供了一组 API 来帮助 XiangShan 比较和验证架构状态。

NEMU 主要有两种配置类型。

In [1]:
%%bash
cd ../ && source env.sh >/dev/null

cd $NEMU_HOME
### If you are building NEMU for the first time,
### you will need to run the following command to configure it.
### Here we have configured it in advance.
# make menuconfig
make clean
make riscv64-xs_defconfig
make -j
make clean-softfloat
make riscv64-xs-ref_defconfig
make -j

rm -rf /home/guest/xs-env/NEMU/build
+ CC src/nemu-main.c
+ CC src/device/alarm.c
+ CC src/device/device.c
+ CC src/device/intr.c
+ CC src/device/uartlite.c
+ CC src/device/plic.c
+ CC src/device/keyboard.c
+ CC src/device/vga.c
+ CC src/device/sdcard.c
+ CC src/cpu/cpu-exec.c
+ CC src/cpu/difftest/dut.c
+ CC src/cpu/difftest/ref.c
+ CC src/cpu/tcache.c
+ CC src/monitor/aligncheck.c
+ CC src/monitor/image_loader.c
+ CC src/monitor/monitor.c
+ CC src/monitor/ui.c
+ CC src/monitor/watchpoint.c
+ CC src/utils/br_log.c
+ CC src/utils/dynamic-config.c
+ CC src/utils/expr.c
+ CC src/utils/filename.c
+ CC src/utils/iqueue.c
+ CC src/utils/log.c
+ CC src/utils/state.c
+ CC src/utils/timer.c
+ CC src/memory/host-tlb.c
+ CC src/memory/paddr.c
+ CC src/memory/vaddr.c
+ CC src/isa/riscv64/clint.c
+ CC src/isa/riscv64/difftest/dut.c
+ CC src/isa/riscv64/difftest/ref.c
+ CC src/isa/riscv64/init.c
+ CC src/isa/riscv64/instr/decode.c
+ CC src/isa/riscv64/instr/fp.c
+ CC src/isa/riscv64/instr/priv/syst

Makefile:40: You have set CONFIG_FLASH_IMG_PATH option, but the specified path '""' does not exist!


+ CC src/nemu-main.c
+ CC src/device/device.c
+ CC src/device/alarm.c
+ CC src/device/flash.c
+ CC src/device/intr.c
+ CC src/cpu/cpu-exec.c
+ CC src/cpu/difftest/dut.c
+ CC src/cpu/difftest/ref.c
+ CC src/cpu/tcache.c
+ CC src/monitor/aligncheck.c
+ CC src/monitor/monitor.c
+ CC src/monitor/image_loader.c
+ CC src/monitor/ui.c
+ CC src/monitor/watchpoint.c
+ CC src/utils/br_log.c
+ CC src/utils/dynamic-config.c
+ CC src/utils/expr.c
+ CC src/utils/filename.c
+ CC src/utils/iqueue.c
+ CC src/utils/log.c
+ CC src/utils/state.c
+ CC src/utils/timer.c
+ CC src/memory/host-tlb.c
+ CC src/memory/paddr.c
+ CC src/memory/vaddr.c
+ CC src/isa/riscv64/clint.c
+ CC src/isa/riscv64/difftest/dut.c
+ CC src/isa/riscv64/difftest/ref.c
+ CC src/isa/riscv64/init.c
+ CC src/isa/riscv64/instr/decode.c
+ CC src/isa/riscv64/instr/fp.c
+ CC src/isa/riscv64/instr/priv/system.c
+ CC src/isa/riscv64/instr/rva/amo.c
+ CC src/isa/riscv64/instr/rvcbo/cbo_impl.c
+ CC src/isa/riscv64/instr/rvk/aes_common.c
+ CC sr

在 NEMU 上运行 Coremark 工作负载。

In [ ]:
%%bash

cd ../ && source env.sh >/dev/null

cd $NEMU_HOME

# 以批模式运行 workload

./build/riscv64-nemu-interpreter -b $AM_HOME/apps/hello/build/hello-riscv64-xs.bin

# Difftest：ISA 协同仿真框架

我们提出了一个 ISA 协同仿真框架 Difftest。其基本流程如下：一旦我们的 RTL 处理器提交指令/更新其他状态，ISA 模拟器就会执行相同的指令。Difftest 会比较 DUT 和 REF 之间的架构状态。如果存在差异，它将停止运行并报告错误。否则，它将继续运行。

在 XiangShan 上运行工作负载，并使用 NEMU 进行差分测试。

In [ ]:
%%bash

cd ../ && source env.sh >/dev/null

cd $NOOP_HOME

./build/emu -i $AM_HOME/apps/hello/build/hello-riscv64-xs.bin --diff $NEMU_HOME/build/riscv64-nemu-interpreter-so 2>perf.out

可以在预编译好的，注入 bug 的香山处理器上运行工作负载，并使用 NEMU 进行差异测试。

In [ ]:
%%bash

cd ../ && source env.sh >/dev/null

cd $NOOP_HOME

./build/emu -i $AM_HOME/apps/hello/build/hello-riscv64-xs.bin --diff $NEMU_HOME/build/riscv64-nemu-interpreter-so

Difftest 检测到错误后转储波形。

In [ ]:
%%bash

cd ../ && source env.sh >/dev/null

cd $NOOP_HOME

./build/emu -i $AM_HOME/apps/hello/build/hello-riscv64-xs.bin \
--diff $NEMU_HOME/build/riscv64-nemu-interpreter-so -b 11000 -e 13000 --dump-wave

ls $NOOP_HOME/build/*.vcd -lh